# Assignment Chapter 2 - DEEP LEARNING [Case #4]
Startup Campus, Indonesia - `Artificial Intelligence (AI)` (Batch 7)
* Task: **CLASSIFICATION**
* DL Framework: **PyTorch**
* Dataset: Credit Card Fraud 2023
* Libraries: Pandas/cuDF, Scikit-learn/cuML, Numpy/cuPy
* Objective: Classify credit fraud transactions using Multilayer Perceptron

`PERSYARATAN` Semua modul (termasuk versi yang sesuai) sudah di-install dengan benar.
<br>`CARA PENGERJAAN` Lengkapi baris kode yang ditandai dengan **#TODO**.
<br>`TARGET PORTFOLIO` Peserta mampu mengklasifikasi transaksi fraud menggunakan *Multilayer Perceptron*

### Import Libraries

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Installing RAPIDS remaining 24.6.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
  Using cached https://pypi.nvidia.com/cudf-cu12/cudf_cu12-24.6.1-cp310-cp310-manylinux_2_28_x86_64.whl (478.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 572.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.7/895.7 kB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 833.8/833.8 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.9/186.9 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import shutil
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

<font color="red">**- - - - MOHON DIPERHATIKAN - - - -**</font>
<br>**Aktifkan GPU sekarang.** Di Google Colab, klik **Runtime > Change Runtime Type**, lalu pilih **T4 GPU**.

### Dataset Loading (CPU vs. GPU)

In [ ]:
from pandas import read_csv as read_by_CPU
from cudf import read_csv as read_by_GPU

In [ ]:
import numpy as np
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
# unzip the file

shutil.unpack_archive('/content/sample_data/dataset_case_04.zip', '/content/sample_data', 'zip')

In [ ]:
# TODO: Impor dataset dengan Pandas, gunakan fungsi "read_by_CPU"

%time data_cpu = read_by_CPU("/content/sample_data/creditcard_2023.csv")

CPU times: user 7.06 s, sys: 394 ms, total: 7.46 s
Wall time: 7.63 s


In [ ]:
# Impor dataset dengan cuDF (Pandas di GPU)
%time data_gpu = read_by_GPU('/content/sample_data/creditcard_2023.csv')

CPU times: user 323 ms, sys: 435 ms, total: 758 ms
Wall time: 1.02 s


In [ ]:
import pandas as pd

data = pd.read_csv("/content/sample_data/creditcard_2023.csv")

In [ ]:
data.head()

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0


In [ ]:
# TODO: Hilangkan kolom ID
data_gpu = data.drop('id', axis=1)
data_gpu.head()


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,0.637735,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,0.529808,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,0.690708,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,0.575231,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,0.968046,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0


### Standardization (CPU vs. GPU)

In [ ]:
from sklearn.preprocessing import StandardScaler as StandardScalerCPU
from cuml.preprocessing import StandardScaler as StandardScalerGPU

In [ ]:
ScalerCPU = StandardScalerCPU()
ScalerGPU = StandardScalerGPU()

arbitrary_features = ["V"+str(i+1) for i in range(27)]

In [ ]:
%%time

data_cpu[arbitrary_features] = ScalerCPU.fit_transform(data_cpu[arbitrary_features].values)
data_cpu["Amount"] = ScalerCPU.fit_transform(data_cpu["Amount"].values.reshape(-1, 1)).squeeze()

CPU times: user 272 ms, sys: 382 ms, total: 654 ms
Wall time: 627 ms


In [ ]:
%%time

data_gpu[arbitrary_features] = ScalerGPU.fit_transform(data_gpu[arbitrary_features].values)
data_gpu["Amount"] = ScalerGPU.fit_transform(data_gpu["Amount"].values.reshape(-1, 1)).squeeze()

CPU times: user 21.4 s, sys: 1.65 s, total: 23 s
Wall time: 23 s


### Train/Test Split (CPU vs. GPU)

In [ ]:
from sklearn.model_selection import train_test_split as splitCPU
from cuml.preprocessing import train_test_split as splitGPU

In [ ]:
# TODO: Tentukan X (features) dan Y (target), gunakan "data_gpu"

X = data_gpu.drop('Class', axis=1)
y = data_gpu['Class']

# Menampilkan bentuk dari X dan Y
print("\nBentuk X:", X.shape)
print("Bentuk Y:", y.shape)


Bentuk X: (568630, 29)
Bentuk Y: (568630,)


In [ ]:
%%time

# TODO: Pecah dataset dengan komposisi 80% train set dan 20% test set, dengan fungsi "splitCPU"
test_size = 0.2
random_state = 0
x_train, x_test, y_train, y_test = splitCPU(X, y, test_size=test_size, random_state=random_state)

print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("\nTest set:")
print("x_test: ", x_test.shape)
print("y_test: ", y_test.shape)


x_train:  (454904, 29)
y_train:  (454904,)

Test set:
x_test:  (113726, 29)
y_test:  (113726,)
CPU times: user 391 ms, sys: 131 ms, total: 522 ms
Wall time: 552 ms


In [ ]:
%%time

# TODO: Lakukan hal yang sama untuk data spliting, tetapi dengan fungsi "splitGPU"
test_size = 0.2
random_state = 0
X_train, X_test, Y_train, Y_test = splitGPU(X, y, test_size=test_size, random_state=random_state)

print("X_train: ", X_train.shape)
print("Y_train: ", Y_train.shape)
print("\nTest set:")
print("X_test: ", X_test.shape)
print("Y_test: ", Y_test.shape)


X_train:  (454904, 29)
Y_train:  (454904,)

Test set:
X_test:  (113726, 29)
Y_test:  (113726,)
CPU times: user 2.3 s, sys: 628 ms, total: 2.93 s
Wall time: 5.14 s


### Convert the dataset into Tensor

In [ ]:
import cupy # Numpy for GPU

In [ ]:
# TODO: Aktifkan GPU (CUDA) sebagai device untuk training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

if device.type == 'cuda':
    print(f'GPU Device Name: {torch.cuda.get_device_name(0)}')
    print(f'Number of GPUs available: {torch.cuda.device_count()}')

Using device: cuda
GPU Device Name: Tesla T4
Number of GPUs available: 1


In [ ]:
import cupy
import torch
from torch.utils.data import TensorDataset

In [ ]:
x_train_tensor = torch.from_numpy(cupy.asnumpy(x_train)).to(device)
y_train_tensor = torch.from_numpy(cupy.asnumpy(y_train)).to(device)

x_test_tensor = torch.from_numpy(cupy.asnumpy(x_test)).to(device)
y_test_tensor = torch.from_numpy(cupy.asnumpy(y_test)).to(device)

Train_tensor = TensorDataset(x_train_tensor, y_train_tensor)
Test_tensor = TensorDataset(x_test_tensor, y_test_tensor)

### Batching the Dataset with PyTorch DataLoader

In [ ]:
# TODO: Tentukan nilai batch
batch_size = 32

Train_dataset = DataLoader(Train_tensor, batch_size=batch_size, shuffle=True)
Test_dataset = DataLoader(Test_tensor, batch_size=batch_size, shuffle=False)

### Model Blueprint

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, input_dim, num_neurons):
        super(FeedForward, self).__init__()
        self.input_dim = input_dim
        self.num_neurons = num_neurons

        self.net = nn.Sequential(
            nn.Linear(self.input_dim, self.num_neurons),
            nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)

    def to(self, device):
        self.net.to(device)
        return self

class Net(nn.Module):
    def __init__(self, in_features, num_layers, num_neurons):
        super(Net, self).__init__()
        self.in_features = in_features
        self.num_layers = num_layers
        self.num_neurons = num_neurons

        self.fc1 = nn.Linear(self.in_features, self.num_neurons)
        self.relu = nn.ReLU()
        self.blocks = [FeedForward(self.num_neurons, self.num_neurons).to(device) \
                       for _ in range(self.num_layers)]
        self.output_layer = nn.Linear(self.num_neurons, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        output = self.relu(self.fc1(x))

        for block in self.blocks:
            output = block(output)
        output = self.sigmoid(self.output_layer(output))

        return output

### Model Hyperparameters and Parameters

In [ ]:
# [ PERTANYAAN ]
# Apa perbedaan hyperparameters dan parameters?

[ ANSWER HERE ]

Parameters:
- Dipelajari oleh model dari data selama proses training.
- Nilai-nilainya diperbarui secara otomatis saat model belajar.
- Disimpan sebagai bagian dari model yang telah dilatih.
  Contoh: bobot (weights) dan bias dalam neural network.

Hyperparameters:
- Diatur manual oleh data scientist sebelum proses training dimulai.
- Mengontrol proses pembelajaran model secara keseluruhan.
- Tidak dipelajari dari data, melainkan ditentukan berdasarkan eksperimen.
  Contoh: learning rate, jumlah layer/neuron, epochs, batch size.

Perbedaan utama:
Cara penentuan: Parameters dipelajari otomatis, hyperparameters diatur manual.
Waktu pengaturan: Parameters diperbarui selama training, hyperparameters ditetapkan sebelumnya.
Peran: Parameters menentukan prediksi model, hyperparameters mengatur proses pembelajaran.
Penyimpanan: Parameters disimpan dalam model, hyperparameters tidak.

In [ ]:
# TODO: Tentukan hyperparameters
epochs = 50
num_layers = 3
num_neurons = 5
learning_rate = 0.01

In [ ]:
# TODO: Tentukan besaran input untuk model
num_inputs = 29

model = Net(in_features=num_inputs, num_layers=num_layers, num_neurons=num_neurons)
model = model.to(device)

In [ ]:
# Set the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

In [ ]:
# Check the number of parameters
print("Number of parameters: {:,}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))
print("Number of trainable parameters: {:,}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

Number of parameters: 156
Number of trainable parameters: 156


In [ ]:
# [ PERTANYAAN ]
# Mengapa total "trainable parameters" sama dengan total keseluruhan parameter?

[ ANSWER HERE ]

Total "trainable parameters" sama dengan total keseluruhan parameter karena:

1. Semua parameter dalam model ini dapat dilatih (trainable). Tidak ada parameter yang dibekukan (frozen) atau ditetapkan sebagai tidak memerlukan gradien.
2. Jika ada lapisan atau parameter tertentu yang diatur sebagai tidak dapat dilatih (misalnya dengan param.requires_grad = False), maka kedua angka tersebut akan berbeda.
3. Model tidak menggunakan teknik seperti transfer learning dengan beberapa lapisan yang dibekukan, atau tidak memiliki parameter khusus yang ditetapkan sebagai tidak dapat dilatih.

### Train the Model

In [ ]:
print("Start training ...")
for epoch in range(epochs):
    train_loss = 0.0
    model.train()

    for data, label in Train_dataset:
        data = data.to(device)
        label = label.squeeze()
        label = label.to(device)
        optimizer.zero_grad()
        output = model.forward(data.float())

        loss = criterion(output.squeeze(), label.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss = train_loss / len(Train_dataset.dataset)
    if(epoch % 10 == 0):
        print('  - Epoch: {} \tTraining_loss: {:.6f}'.format(epoch, train_loss))

Start training ...
  - Epoch: 0 	Training_loss: 0.003655
  - Epoch: 10 	Training_loss: 0.001761
  - Epoch: 20 	Training_loss: 0.001632
  - Epoch: 30 	Training_loss: 0.001611
  - Epoch: 40 	Training_loss: 0.001626


### Model ACCURACY should reach >= 95%

In [ ]:
# TODO: Jika akurasi masih dibawah 95%, silakan lakukan fine-tuning

In [ ]:
correct_preds = 0
total_samples = 0

with torch.no_grad():
    for data, labels in Test_dataset:
        labels = labels.squeeze()
        output = model.forward(data.float())
        output = output.squeeze(1)

        predictions = (output >= 0.5).float()
        correct_preds += (predictions == labels).sum().item()
        total_samples += labels.numel()

accuracy = correct_preds / total_samples
print("Model accuracy: {:.2f}%".format(accuracy*100))

Model accuracy: 98.60%


### Scoring
Total `#TODO` = 12
<br>Checklist:

- [ ] Impor dataset dengan Pandas, gunakan fungsi "read_by_CPU"
- [ ] Hilangkan kolom ID
- [ ] Tentukan X (features) dan Y (target), gunakan "data_gpu"
- [ ] Pecah dataset dengan komposisi 80% train set dan 20% test set, dengan fungsi "splitCPU"
- [ ] Lakukan hal yang sama untuk data spliting, tetapi dengan fungsi "splitGPU"
- [ ] Aktifkan GPU (CUDA) sebagai device untuk training
- [ ] Tentukan nilai batch
- [ ] PERTANYAAN: Apa perbedaan hyperparameters dan parameters?
- [ ] Tentukan hyperparameters
- [ ] Tentukan besaran input untuk model
- [ ] PERTANYAAN: Mengapa total "trainable parameters" sama dengan total keseluruhan parameter?
- [ ] Jika akurasi masih dibawah 95%, silakan lakukan fine-tuning

### Additional readings
- N/A

### Copyright © 2024 Startup Campus, Indonesia
* Prepared by **Nicholas Dominic, M.Kom.** [(profile)](https://linkedin.com/in/nicholas-dominic)
* You may **NOT** use this file except there is written permission from PT. Kampus Merdeka Belajar (Startup Campus).
* Please address your questions to mentors.